This notebook contains a short way to only verify that the Laplacians in the relevant degrees have non-trivial rank.

# Setup

In [53]:
using Pkg
Pkg.activate(@__DIR__)
using LinearAlgebra
ENV["JULIA_NUM_THREADS"] = Sys.CPU_THREADS÷2
LinearAlgebra.BLAS.set_num_threads(Sys.CPU_THREADS÷2)

using Groups
using LowCohomologySOS
using SLnCohomology
using AbstractAlgebra
using BlockArrays
using LinearAlgebraX
using JSON
using Permutations
using SparseArrays

  Activating project at `/mnt/c/Git/SLnCohomology`


In [54]:
n, degree = 3, 3
# n, degree = 4, 6 #uncomment if you want to do the computations for n=4

(3, 3)

In [55]:
sln = MatrixGroups.SpecialLinearGroup{n}(Int8)

special linear group of 3×3 matrices over Int8

# Voronoi tesselation
We compute the Voronoi tesselation, together with boundaries of cells and their stabilisers.

In [71]:
cells_sln = SLnCohomology.cells_sln(n)
oriented_cells_sln = SLnCohomology.oriented_cells_dict(cells_sln)
boundaries = SLnCohomology.boundaries_dict(oriented_cells_sln)
stabilisers = SLnCohomology.stabilisers_dict(oriented_cells_sln)

I computed minimal vectors.
I'm computing in dimension 4
I'm computing in dimension 3
I'm computing in dimension 2
Dimension 5
Cell number 1
Dimension 4
Cell number 1
Dimension 2
Cell number 1
Dimension 3
Cell number 1
Cell number 2


Dict{Any, Any} with 4 entries:
  5 => Any[Any[([0 -1 0; 0 0 -1; 1 1 1], 1), ([0 1 1; 0 -1 0; 1 1 0], 1), ([0 0…
  4 => Any[Any[([-1 0 0; 1 1 1; 0 0 -1], 1), ([1 0 0; -1 -1 0; 0 0 -1], 1), ([-…
  2 => Any[Any[([-1 0 0; 2 1 1; 0 0 -1], -1), ([-1 -1 -1; 2 2 1; 0 -1 0], -1), …
  3 => Any[Any[([0 1 1; -1 -2 -1; 2 2 1], -1), ([0 1 1; 1 0 -1; -2 -1 0], 1), (…

In [57]:
cell_degrees = sort([first(x) for x in boundaries]) # the degrees where the Voronoi complex has cells intersecting the interior
min_degree = cell_degrees[1]
max_degree = cell_degrees[end]
relevant_degrees_differential = [degree,degree+1] # to compute the Laplacian Delta_n, we need the differentials partial_n and partial_{n+1}
relevant_degrees_group_ring = [degree-1,degree,degree+1] # we need the group rings in one degree higher, as the images of the adjoints

3-element Vector{Int64}:
 2
 3
 4

# Computing the Laplacian over the group ring
To store the Laplacian, we first compute the supports (i.e. half_bases) for the group rings.

In [72]:
# To compute the Laplacians we just add to half_basis the coset elements appearing in the differentials.
d_union = Dict(k=>[one(sln)] for k in cell_degrees)
for k in relevant_degrees_group_ring
    if !(k == min_degree) # in min_degree, all boundaries are trivial, so nothing needs to be added
        for i in eachindex(boundaries[k])
            for j in eachindex(boundaries[k][i])
                coset = boundaries[k][i][j]["orbit_coset_with_orientation"]
                d_union[k] = union(
                    d_union[k], 
                    [SLnCohomology.gelt_from_matrix(M,sln) for (M,eta) in coset]
                )
            end
        end
    end
end
half_basis_Δ = Dict()
for k in relevant_degrees_group_ring
    if k == max_degree
        half_basis_Δ[k] = d_union[k]
        half_basis_Δ[k] = unique([half_basis_Δ[k];inv.(half_basis_Δ[k])])
    else
        half_basis_Δ[k] = union(d_union[k], d_union[k+1])
        half_basis_Δ[k] = unique([half_basis_Δ[k];inv.(half_basis_Δ[k])])
    end
end

# Compute the group rings (with standard multiplication by convolution: (1+g)(1+h)=1+g+h+gh)
RG_Δ = Dict()
for k in relevant_degrees_group_ring
    RG_Δ[k] = LowCohomologySOS.group_ring(sln, half_basis_Δ[k])
end


We then compute the differentials as matrices over the group ring. We store them as matrices over Rationals to perform exact operations.

In [73]:
cells_number = Dict(k=>length(cells_sln[k]) for k in cell_degrees) # number of cell orbits in degree k
cells_number[min_degree-1] = 0
dx = Dict([(k,i,j) => 0//1*zero(RG_Δ[k-1]) for k in relevant_degrees_differential for i in 1:cells_number[k-1] for j in 1:cells_number[k]]) # This is empty for min_degree
for k in relevant_degrees_differential
    for j in eachindex(boundaries[k]) # each j corresponds to a k-cell sigma
        for tauprime in boundaries[k][j] # each tau' is a facet of sigma
            coset_orient = [(SLnCohomology.gelt_from_matrix(M,sln),eta) for (M,eta) in tauprime["orbit_coset_with_orientation"]]
            # coset_orient is G(tau,tau'); gelt_from_matrix(M,sln) is an element g sending tau to tau', eta is eta(tau,tau',g)
            summand_in_RG = tauprime["sign"]*SLnCohomology.averaged_rep(coset_orient,RG_Δ[k-1]) 
            # summand_in_RG is x_{tau'}; tauprime["sign"] is eta(tau',sigma,1)
            dx[k,tauprime["orbit_standard_cell"],j] += summand_in_RG 
            # dx[k,tauprime["orbit_standard_cell"],j] is partial_{sigma,tau}; tauprime["orbit_standard_cell"] is tau
        end
    end 
end
d = Dict()
for k in relevant_degrees_differential
    d[k] = [dx[k,i,j] for i in 1:cells_number[k-1], j in 1:cells_number[k]]
end

The averaged stabilisers, needed for the chain group $V_{degree}$:

In [74]:
stab_part_dim = Dict()
# Convert the stabilisers to signed subgroups of SL(n,ℤ)
m_stabs = [
    [(SLnCohomology.gelt_from_matrix(M,sln),eta) for (M,eta) in stab] for stab in stabilisers[degree]
]
stab_part_dim[degree] = [
    i == j ? one(RG_Δ[degree])-SLnCohomology.averaged_rep(m_stabs[i],RG_Δ[degree]) : zero(zero(RG_Δ[degree])) 
    for i in 1:cells_number[degree],j in 1:cells_number[degree]
]
# The above also verifies that the stabilisers' elements belong to the half bases.

2×2 Matrix{StarAlgebras.AlgebraElement{StarAlgebra{Groups.MatrixGroups.SpecialLinearGroup{3, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}}, FPGroupElement{Groups.MatrixGroups.SpecialLinearGroup{3, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}}, …}, …}, T} where T}:
 11//12·(id) +1//12·E₁₂^-1*E₂₁*E₁₂^-1*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₂₃*E₁₃^-1  -1//12·E₂₁^-2*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^-1*E₂₃^-1*E₃₂*E₂₃^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₁₃*E₁₂  -1//12·E₃₁^-1*E₂₃^-1*E₃₂*E₂₃^-1*E₂₃^2*E₁₃*E₁₂  +1//12·E₁₂^-1*E₂₁*E₁₂^-1*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^2*E₃₂^-2*E₂₃*E₁₃*E₁₂^2  +1//12·E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₂₃^-1*E₃₂*E₂₃^-1*E₁₃*E₁₂  -1//12·E₂₁^-2*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^-2*E₃₂^3*E₂₃^-2*E₃₂*E₂₃^-2  +1//12·E₂₁^-2*E₃₁^2*E₂₃^-1*E₃₂*E₂₃^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₂₃^2*E₁₃*E₁₂  -1//12·E₁₂^-1*E₂₁*E₁₂^-1*E₃₂^-1*E₂₃*E₁₃*E₁₂^2  +1//12·E₂₁^-2*E₃₁

The Laplacian is then computed as follows.

In [75]:
d_k = LowCohomologySOS.embed.(identity, d[degree], Ref(RG_Δ[degree]))
d_k_plus_1 = LowCohomologySOS.embed.(identity, d[degree+1], Ref(RG_Δ[degree]))
Δ = d_k'*d_k+d_k_plus_1*d_k_plus_1'+stab_part_dim[degree] # here we use that partial_sigma,tau*v_sigma = partial_sigma

2×2 Matrix{StarAlgebras.AlgebraElement{StarAlgebra{Groups.MatrixGroups.SpecialLinearGroup{3, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}}, FPGroupElement{Groups.MatrixGroups.SpecialLinearGroup{3, Int8, DataType, Alphabet{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}, Vector{Groups.MatrixGroups.ElementaryMatrix{3, Int8}}}, …}, …}, Rational{Int64}, SparseVector{Rational{Int64}, Int64}}}:
 11//8·(id) -1//24·E₁₂^-1*E₂₁*E₁₂^-1*E₂₃*E₁₃*E₁₂^2  -1//72·E₁₂^-1*E₂₁*E₁₂^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₂₃  -1//72·E₁₂^-1*E₂₁*E₁₂^-1*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₂₃*E₁₃^2*E₁₂^2  -3//8·E₁₂^-1*E₂₁*E₁₂^-1*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₂₃*E₁₃^-1  +1//72·E₂₁^-2*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^-1*E₂₃^-1*E₃₂*E₂₃^-1*E₁₂  +3//8·E₂₁^-2*E₂₁*E₁₂^-2*E₂₁*E₁₂^-2*E₃₁^-1*E₂₃^-1*E₃₂*E₂₃^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₁₃*E₁₂  +1//24·E₂₁^-2*E₃₁*E₂₃^-1*E₃₂*E₂₃^-1*E₃₂*E₂₃^-2*E₃₂*E₂₃^-2*E₂₃^2*E₁₃*E₁₂  +1//72·E₂₁^-2*E₃₁*E₂₃^-1*E₃₂*E₂₃^-1*E₂₃^2*E₁₂  +3

# Evaluating the Laplacian for specific representations

We now construct the representations described in the article and evaluate the Laplacian on them.

The representation is induced up from $\operatorname{SL}_n(\mathbb Z/p)$ for the following $p$.

In [65]:
p = (n == 3 ? 3 : 2)

3

We compute an orthogonal representation without invariant vectors of a subgroup H of $\operatorname{SL}_n(\mathbb Z/p)$.

In [76]:
subgroup_rep, deg = SLnCohomology.flip_permutation_representation(n,p)
H = keys(subgroup_rep)
no_inv_subspace = SLnCohomology.no_inv_subspace(H,subgroup_rep)
@assert length(no_inv_subspace) == deg # this means that we have no invariant vectors

Induce the repesentation of H to $\operatorname{SL}_n(\mathbb Z/p)$.

In [77]:
sl_n_p_matrices = SLnCohomology.sl_n_p(n,p)
support = SLnCohomology.laplacians_support(Dict(degree => Δ),p)
coset_data = SLnCohomology.coset_data(H,sl_n_p_matrices,p)
pi = SLnCohomology.ind_rep_dict(support,subgroup_rep,coset_data,deg,p)

Dict{Any, Any} with 760 entries:
  Int8[1 0 1; 0 1 1; 2 1 1] => sparse([136, 90, 52, 128, 36, 106, 152, 63, 79, …
  Int8[2 0 0; 1 1 1; 1 0 2] => sparse([133, 96, 32, 150, 69, 59, 126, 42, 86, 1…
  Int8[0 1 0; 2 2 2; 0 0 1] => sparse([100, 91, 82, 73, 64, 55, 46, 37, 28, 92 …
  Int8[2 2 0; 1 2 0; 2 1 2] => sparse([80, 123, 45, 53, 131, 72, 107, 147, 99, …
  Int8[1 1 0; 2 1 2; 1 0 1] => sparse([20, 144, 24, 26, 156, 27, 23, 143, 21, 1…
  Int8[1 1 1; 2 0 2; 0 0 1] => sparse([53, 45, 34, 107, 99, 88, 80, 72, 61, 54 …
  Int8[2 1 1; 1 1 2; 2 0 2] => sparse([99, 123, 53, 72, 147, 80, 45, 131, 107, …
  Int8[0 1 1; 2 2 2; 0 0 1] => sparse([101, 92, 83, 74, 65, 56, 47, 38, 29, 93 …
  Int8[2 1 0; 0 1 2; 1 0 1] => sparse([52, 136, 90, 79, 152, 63, 106, 128, 36, …
  Int8[1 2 0; 2 2 1; 2 2 2] => sparse([96, 150, 86, 69, 126, 32, 42, 133, 59, 1…
  Int8[1 2 0; 0 2 0; 2 1 2] => sparse([7, 139, 3, 1, 140, 9, 4, 154, 6, 104  … …
  Int8[0 2 1; 1 1 0; 2 1 2] => sparse([52, 128, 98, 106, 136, 44, 79, 152, 7

Evaluate the Laplacian on this representation.

In [78]:
piΔ = vcat(
    [
        hcat([SLnCohomology.representing_matrix(Δ[i,j],pi, p) for j in 1:size(Δ)[2]]...)
        for i in 1:size(Δ)[1]
    ]...
)

312×312 Matrix{Rational{Int64}}:
 29//24   -1//72   0//1   -5//24  -1//36  …  0//1   0//1    0//1    0//1
 -1//72  107//72   1//36   1//72   0//1      0//1   0//1    0//1    0//1
  0//1     1//36  19//12   0//1    0//1      0//1   0//1    0//1    0//1
 -5//24    1//72   0//1   29//24   1//36     0//1   0//1    0//1    0//1
 -1//36    0//1    0//1    1//36   4//3      0//1   0//1    0//1    0//1
  0//1    -1//9    0//1    0//1    0//1   …  0//1  -1//36   1//18  -1//36
  0//1     0//1    0//1    0//1    0//1      0//1   0//1    0//1    0//1
  1//24   -1//72  -1//9   -1//24   5//36     0//1   1//36  -1//6    1//36
  0//1     0//1    1//9    0//1   -1//12     0//1   0//1   -2//9    0//1
  0//1     0//1    0//1    0//1    0//1      0//1   0//1    0//1    0//1
   ⋮                                      ⋱                  ⋮     
  0//1     0//1    0//1    0//1    0//1      0//1   1//6   -1//3    1//6
  0//1     0//1    0//1    0//1    0//1      0//1   0//1    0//1    0//1
  0//1     0//1    0/

Compute the rank of reduced cohomology.

In [81]:
println("rank H^"*string(div(n*(n-1),2)+n-degree-1)*"(SL_n(Z);pi) = "*string(size(piΔ)[1]-rankx(piΔ)))


rank H^2(SL_n(Z);pi) = 4
